In [1]:
# !pip install unidecode

In [2]:
# !conda install --yes -c conda-forge unidecode

In [3]:
import re, os, sys, requests, time, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import bs4
from bs4 import BeautifulSoup
from unidecode import unidecode
from random import randint

In [4]:
print(f'O notebook está na pasta: {os.getcwd()}')

O notebook está na pasta: C:\Users\viniz\OneDrive\Área de Trabalho\Vinícius\Insper\Matérias\Eletivas\Análise de Texto e Fontes Desestruturadas e Web\Projeto\projeto-fontes-desestruturadas-e-web


In [5]:
# ===== APENAS PARA A PRIMEIRA PÁGINA DE CADA CAPITAL POR ENQUANTO =====
'''
ANOTAÇÕES

- Fazer com que o código pesquise no Google o endereço de cada local e
retorne separadamente bairro, número, tipo de logradouro e
nome do logradouro;
OU
- Fazer com que o código pesquise no Google o endereço de cada local e
retorne a latitude e longitude, plotando num mapa ao decorrer do projeto;

- Fazer o Web Crawling de todas as páginas disponíveis para cada cidade
(é esperado que demore, pois cidades como São Paulo possuem milhões de
imóveis listados, talvez seja necessário considerar um limite máximo
como 10 mil imóveis por cidade);

- Não fazer a pesquisa com base nas ordenações disponíveis;

- Fazer com que o código acesse imóvel por imóvel e retone informações
booleanas adicionais do imóvel, por exemplo, se possui elevador (quantidade?),
se possui piscina (quantidade?), se possui academia, se possui espaços
para festas gerais;
''';

In [6]:
class Projeto:
    def __init__(self):
        self.posicoes = []
        self.enderecos = []
        self.areas = []
        self.quartos = []
        self.banheiros = []
        self.vagas = []
        self.precos = []
        self.conds = []
        self.iptus = []
        self.descricoes = []
        self.lista_status = []
        self.paginas = []

    def get_urls(self) -> list:
        urls = []
        uf_capital = {}
        
        url_capitais = 'https://www.todamateria.com.br/capitais-do-brasil/'
        resposta = requests.get(url_capitais)
        resposta.encoding = 'utf-8'
        
        soup = BeautifulSoup(resposta.text, 'html.parser')
        
        tabela = soup.find('tbody')
        linhas = tabela.find_all('tr')
        for linha in linhas:
            uf = linha.find_all('td')[2].text
            uf = uf.lower()
            capital = linha.find_all('td')[0].text
            capital = capital.lower()
            capital = capital.split()
            capital = '-'.join(capital)
            capital = unidecode(capital)
            uf_capital[uf] = capital
        
        for uf, capital in uf_capital.items():
            url = 'https://www.zapimoveis.com.br'
            uri = '/venda/imoveis/'
            uri += f'{uf}+{capital}'
            url += uri
            urls.append(url)
        
        return urls
        
    def endereco(self, produto):
        try:
            endereco = produto.find('h2', attrs={'itemprop': 'address'}).text
            endereco = endereco.strip('\n ')
            self.enderecos.append(endereco)

        except:
            self.enderecos.append(None)

    def area(self, produto):
        try:
            area = produto.find('li', class_='feature__item text-small js-areas').text
            area = area.strip('\n ')
            area = area.replace('m²', '')
            self.areas.append(area)
        except:
            self.areas.append(None)
        
    def qtd_quartos(self, produto):
        try:
            quarto = produto.find('span', attrs={'itemprop': 'numberOfRooms'}).text
            quarto = quarto.strip('\n ') # Pode ser algo do tipo: '1 - 3'
            self.quartos.append(quarto)
        except:
            self.quartos.append(None)
        
    def qtd_banheiros(self, produto):
        try:
            banheiro = produto.find('span', attrs={'itemprop': 'numberOfBathroomsTotal'}).text
            banheiro = banheiro.strip('\n ') # Pode ser algo do tipo: '1 - 3'
            self.banheiros.append(banheiro)
        except:
            self.banheiros.append(None)
        
    def qtd_vagas(self, produto):
        try:
            vaga = produto.find('li', class_='feature__item text-small js-parking-spaces').text
            vaga = vaga.strip('\n ') # Pode ser algo do tipo: '1 - 3'
            self.vagas.append(vaga)
        except:
            self.vagas.append(None)
        
    def condominio(self, produto):
        try:
            cond = produto.find('li', class_='card-price__item condominium text-regular').text
            self.conds.append(''.join(re.findall(r'\d+', cond)))
        except:
            self.conds.append(None)
        
    def iptu(self, produto):
        try:
            iptu = produto.find('li', class_='card-price__item iptu text-regular').text
            self.iptus.append(''.join(re.findall(r'\d+', iptu)))
        except:
            self.iptus.append(None)
        
    def descricao(self, produto):
        try:
            descricao = produto.find('span', attrs={'itemprop': 'description'}).text
            self.descricoes.append(descricao)
        except:
            self.descricoes.append(None)
        
    def status(self, produto):
        try:
            status = produto.find('small').text
            self.lista_status.append(status)
        except:
            self.lista_status.append('Comum')
        
    def preco(self, produto):
        preco = produto.find('p', class_='simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left').text
        self.precos.append(''.join(re.findall(r'\d+', preco)))
        
    def everything(self, produto):
        self.endereco(produto)
        self.area(produto)
        self.qtd_quartos(produto)
        self.qtd_banheiros(produto)
        self.qtd_vagas(produto)
        self.condominio(produto)
        self.iptu(produto)
        self.descricao(produto)
        self.status(produto)
        self.preco(produto)
        
    def main(self):
        urls = self.get_urls()
        for url in urls:
            posicao = 1
            pagina = 1
            
            n1, n2, n3 = randint(0, 36), randint(0, 142), randint(0, 35)
            headers = {'user-agent':
                       f'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.{n1} (KHTML, like Gecko) Chrome/75.0.3770.{n2} Safari/537.{n3}'
            }
            
            resposta = requests.get(url, headers=headers)
            resposta.encoding = 'utf-8'

            soup = BeautifulSoup(resposta.text, 'html.parser')

            lista_prod = soup.find('div', class_='listings__container')
            produtos = lista_prod.find_all('div', class_='card-container js-listing-card')
            
            for produto in produtos:
                self.posicoes.append(posicao)
                posicao += 1
                
                self.everything(produto)
                
                self.paginas.append(pagina)
                
            pagina += 1
        
        dicionario = {'Posição': self.posicoes,
                      'Endereço': self.enderecos,
                      'Área': self.areas,
                      'Quartos': self.quartos,
                      'Banheiros': self.banheiros,
                      'Vagas': self.vagas,
                      'Condomínio': self.conds,
                      'IPTU': self.iptus,
                      'Descrição': self.descricoes,
                      'Status': self.lista_status,
                      'Página': self.paginas,
                      'Preço': self.precos}
        
        df = pd.DataFrame(dicionario)
        
        return df

In [7]:
projeto = Projeto()
df = projeto.main()
df.head()

,Posição,Endereço,Área,Quartos,Banheiros,Vagas,Condomínio,IPTU,Descrição,Status,Página,Preço
0,1,"Avenida Recanto Verde, Residencial Petropolis",114,3,2,2,None,None,O sobrado no bairro Primavera possui 114 metro...,Destaque,1,495000
1,2,"Rua C, LBA Vila Betel ll",336,2,2,1,None,None,?Casa na Vila Betel ! Vende-se uma casa no co...,Destaque,1,185000
2,3,"Rua Belo Horizonte, Ivete Vargas",170,4,2,1,None,None,Vende-se uma casa no Ivete Vargas. A casa poss...,Destaque,1,245000
3,4,"Avenida Getúlio Vargas, Centro",250,3,None,2,None,None,None,Destaque,1,500000
4,5,"Estrada da Floresta, Floresta Sul",62,3,2,1,280,None,O apartamento no bairro Floresta Sul tem 62 me...,Destaque,1,225000


In [8]:
cwd = str(os.getcwd())
_, _, files = next(os.walk(cwd))
file_count = len(files)
print(file_count)

df.to_csv(f'data{file_count - 1}.csv')

10


In [9]:
# Tipos de status
df['Status'].value_counts().index

Index(['Destaque', 'Comum', 'Super destaque', 'Em construção', 'Na planta',
       'Pronto para morar'],
      dtype='object')

In [10]:
# Quantidade bruta de imóveis por tipo de status
df['Status'].value_counts()

Destaque             259
Comum                245
Super destaque        75
Em construção         32
Na planta             20
Pronto para morar     17
Name: Status, dtype: int64

In [11]:
# Quantidade relativa de imóveis por tipo de status
df['Status'].value_counts(True).round(4)*100

Destaque             39.97
Comum                37.81
Super destaque       11.57
Em construção         4.94
Na planta             3.09
Pronto para morar     2.62
Name: Status, dtype: float64

In [12]:
super_mask = df['Status'] == 'Super destaque'
super_df = df.loc[super_mask, :]
super_df

,Posição,Endereço,Área,Quartos,Banheiros,Vagas,Condomínio,IPTU,Descrição,Status,Página,Preço
27,4,"Rua Antenor Marinho de Melo, Gruta de Lourdes",740,None,None,None,None,None,Ginásio de esporte com 740 Mt de área construída,Super destaque,1,1000000
28,5,"Avenida Doutor Antônio Gouveia, Pajuçara",380,4,6,4,1300,5500,"Cobertura Garden, na Pajuçara, beira mar, com ...",Super destaque,1,3500000
29,6,"Rua Ailton Torres, Serraria",68,3,2,1,350,499,De 270mil por 220mil Na MANGABEIRAS 3quarto...,Super destaque,1,220000
48,1,"Alameda das Estrelas, Muruci Fazendinha",332,None,None,None,350,None,"Lotes medindo mais de 332,00m² em excelente l...",Super destaque,1,250000
75,4,"Avenida Coronel Teixeira, Ponta Negra",49,2,2,1,None,None,O Estilo Ponta Negra tem uma infraestrutura gr...,Super destaque,1,260000
...,...,...,...,...,...,...,...,...,...,...,...,...
604,5,"Avenida Dulce Diniz, Luzia",80,3,2,1,410,1100,Virtuosi Residence / Reformado / Moveis Planej...,Super destaque,1,415000
605,6,"Rua Matilde Silva Lima, Luzia",68,3,1,1,302,None,"Apartamento térreo amplo com sala, hall, três ...",Super destaque,1,155000
627,4,"Plano Diretor Sul, Palmas",368,None,None,None,None,None,Prédio Comercial a venda Localização mais que...,Super destaque,1,3300000
628,5,"Plano Diretor Sul, Palmas",586,4,5,5,None,None,A melhor e maior Cobertura Duplex de Palmas - ...,Super destaque,1,4970001


In [13]:
destaque_mask = df['Status'] == 'Destaque'
destaque_df = df.loc[destaque_mask, :]
destaque_df

,Posição,Endereço,Área,Quartos,Banheiros,Vagas,Condomínio,IPTU,Descrição,Status,Página,Preço
0,1,"Avenida Recanto Verde, Residencial Petropolis",114,3,2,2,None,None,O sobrado no bairro Primavera possui 114 metro...,Destaque,1,495000
1,2,"Rua C, LBA Vila Betel ll",336,2,2,1,None,None,?Casa na Vila Betel ! Vende-se uma casa no co...,Destaque,1,185000
2,3,"Rua Belo Horizonte, Ivete Vargas",170,4,2,1,None,None,Vende-se uma casa no Ivete Vargas. A casa poss...,Destaque,1,245000
3,4,"Avenida Getúlio Vargas, Centro",250,3,None,2,None,None,None,Destaque,1,500000
4,5,"Estrada da Floresta, Floresta Sul",62,3,2,1,280,None,O apartamento no bairro Floresta Sul tem 62 me...,Destaque,1,225000
...,...,...,...,...,...,...,...,...,...,...,...,...
643,20,"Rua 17, Taquarussu",300,4,4,6,None,None,"🧡 Linda chácara , às margens da PALMAS -TO a T...",Destaque,1,2200000
644,21,"Plano Diretor Norte, Palmas",68,3,2,2,None,None,Vende-se apartamento de 3/4 Sendo 68 m² muito...,Destaque,1,245000
645,22,"Plano Diretor Sul, Palmas",140,3,3,3,None,None,Casa nova na quadra 309 Sul com 140m2 de área ...,Destaque,1,600000
646,23,"ARSO 42 Avenida LO 9, Plano Diretor Sul",73,3,2,2,None,None,Vendo agio referente o valor já pago da entrad...,Destaque,1,110000


In [14]:
comum_mask = df['Status'] == 'Comum'
comum_df = df.loc[comum_mask, :]
comum_df

,Posição,Endereço,Área,Quartos,Banheiros,Vagas,Condomínio,IPTU,Descrição,Status,Página,Preço
5,6,"Rua Acre, Vila da Amizade",50,2,1,1,None,None,"Aluga-se apartamento no Residencial Villacre, ...",Comum,1,150000
9,10,"Estrada do Calafate, Calafate",50,2,1,1,None,None,Vendo esse imóvel em condomínio fechado. Apart...,Comum,1,89000
11,12,"Avenida Ceará, Dom Giocondo",110,3,3,2,None,None,A casa está localizado no bairro Dom Giocondo ...,Comum,1,400000
14,15,"Rua São Mateus, Nova Esperanca",80,3,2,2,None,None,O apartamento está localizado no bairro Nova E...,Comum,1,480000
18,19,"Rua Antônio Carlos, Sao Francisco",191,3,2,3,None,None,A casa no bairro São Francisco tem 191 metros ...,Comum,1,335000
...,...,...,...,...,...,...,...,...,...,...,...,...
632,9,"Plano Diretor Sul, Palmas",103,3,3,2,None,None,1003 Sul; Lote: 180 m² Área construída: 103 m...,Comum,1,385000
635,12,"Plano Diretor Norte, Palmas",87,3,2,2,None,None,Vendo Ágio de apartamento novo Pronto para Mor...,Comum,1,80000
636,13,"Plano Diretor Sul, Palmas",113,3,4,2,None,None,Área Privativa: 113m² 3 Suítes plenas Sala am...,Comum,1,670000
640,17,"Jardim Aureny III, Palmas",120,3,2,2,None,None,Vendo casa de esquina localizada no Aureny III...,Comum,1,190000


In [15]:
construcao_mask = df['Status'] == 'Em construção'
construcao_df = df.loc[construcao_mask, :]
construcao_df

,Posição,Endereço,Área,Quartos,Banheiros,Vagas,Condomínio,IPTU,Descrição,Status,Página,Preço
26,3,"Rua Valdo Omena, Ponta Verde",37 - 79,1 - 2,1,1 - 2,None,None,Venha conhecer o Smart Stay.,Em construção,1,
98,3,"Rua Parambu, Santa Teresa",58,2,2,1,None,None,Residencial Santorini – BRF Realizações Jardi...,Em construção,1,395000
121,2,"Rua Gonçalves Ledo, Centro",100,3,4,3,None,None,Venha morar onde o melhor da vida acontece! R...,Em construção,1,759000
122,3,"Rua Gonçalves Ledo, Centro",100,3,4,2,None,None,LAZER COMPLETO Piscina adulto e infantil com d...,Em construção,1,
144,1,"SQNW 302 Bloco C, Setor Noroeste",148,4,6,3,None,None,"Apartamento 4 quartos, sendo 4 suites à venda ...",Em construção,1,2282226
145,2,"EQS 414/415, Asa Sul",67 - 188,2 - 3,None,None,None,None,O Reserva Parque Clube é assim: a combinação p...,Em construção,1,
192,1,"Rua 22, Setor Oeste",219,4,6,4,None,None,Adagio - Opção de 3 suítes com possibilidade d...,Em construção,1,1684727
193,2,"Rua 22, Setor Oeste",148,3 - 4,5,3,None,None,Moderato - Sala com mega esquadria e balcão qu...,Em construção,1,1088094
194,3,"Rua 22, Setor Oeste",135,2 - 3,5,2,None,None,Vivace - Layout da sala com possibilidade de a...,Em construção,1,
240,1,"Avenida Ipiranga, Porto",35 - 44,1 - 2,1,1,None,None,"Apartamentos de 2 quartos no Porto, em condomí...",Em construção,1,217990


In [16]:
planta_mask = df['Status'] == 'Na planta'
planta_df = df.loc[planta_mask, :]
planta_df

,Posição,Endereço,Área,Quartos,Banheiros,Vagas,Condomínio,IPTU,Descrição,Status,Página,Preço
24,1,"Avenida Comendador Gustavo Paiva, Mangabeiras",39 - 71,None,2,None,None,None,THE ICON PARK OFFICE Um EMPRESARIAL ao lado d...,Na planta,1,297394
73,2,"Rua Fulvio Pennanchi, Planalto",38 - 43,1 - 2,1,1,None,None,"Lazer equipado, Previsão para medição de água ...",Na planta,1,176490
97,2,"Rua Doutor Genésio Salles, Vila Laura",61 - 89,2 - 3,2 - 3,1 - 2,None,None,O melhor da Vila Laura para você. Escolha mais...,Na planta,1,387480
120,1,"Rua Nogueira Acioli, Joaquim Tavora",51 - 65,2 - 3,2 - 3,1 - 2,None,None,"Modernidade, conforto e acessibilidade. Elemen...",Na planta,1,
168,1,"Rua das Marrecas, Centro",32 - 66,1,1,None,None,None,Hub Coliving. Studios com área de convivência ...,Na planta,1,
170,3,"Rua Irineu Marinho, Centro",35,1 - 2,1,None,None,None,Cores do Rio Centro da Cidade RJ: É o primeiro...,Na planta,1,272000
216,1,"Avenida Dom José Delgado, Alemanha",39 - 42,2,1,1,None,None,"Apartamentos de 2 quartos no Alemanha, em cond...",Na planta,1,163990
218,3,"Avenida dos Holandeses, Ponta D'areia",81 - 90,3,1,2,None,None,Imagine alcançar outro nível na sua qualidade ...,Na planta,1,889300
338,3,"Avenida Pombal, Manaíra",41,1 - 3,1 - 2,1 - 2,None,None,"Localizado em Manaíra, a 300 metros do mar, o ...",Na planta,1,263000
361,2,"Avenida Visconde de Guarapuava, Centro",32 - 72,1 - 2,1 - 2,1 - 3,None,None,"Conheça o Mont Tannat, o novo empreendimento d...",Na planta,1,258013


In [17]:
pronto_mask = df['Status'] == 'Pronto para morar'
pronto_df = df.loc[pronto_mask, :]
pronto_df

,Posição,Endereço,Área,Quartos,Banheiros,Vagas,Condomínio,IPTU,Descrição,Status,Página,Preço
25,2,"Rua Doutora Rosa Cabús, Jatiúca",40 - 59,1 - 2,1 - 2,1,None,None,"Viva o presente, invista no futuro! Viver é g...",Pronto para morar,1,
72,1,"Avenida Coronel Teixeira, Ponta Negra",66 - 88,2 - 3,2 - 3,1,None,None,Pronto para morar ao lado do Shopping Ponta Ne...,Pronto para morar,1,340000
74,3,"Avenida Coronel Teixeira, Ponta Negra",36 - 49,1 - 2,1,1,None,None,A região da Ponta Negra é um dos pontos mais n...,Pronto para morar,1,
96,1,"3ª Travessa Paripiranga, Bairro da Paz",53 - 71,2 - 3,2,1,None,None,Pronto para morar pertinho da praia!Pronto par...,Pronto para morar,1,
146,3,"CLSW 105 Bloco A, Setor Sudoeste",97,None,3,None,None,None,"Lojas comerciais à venda em Sudoeste, Brasília...",Pronto para morar,1,1965592
169,2,"Rua Ubaldino do Amaral, Centro",38 - 69,1 - 2,1 - 2,1,None,None,"A vida prática e moderna, como nas cidades mai...",Pronto para morar,1,349900
217,2,"Avenida da História, Cohafuma",28 - 38,1,1,1,None,None,residencial cohafuma: do tamanho certo para vo...,Pronto para morar,1,145000
290,3,"Avenida Álvares Cabral, Lourdes",78 - 83,3,3,2 - 3,None,None,"Lumière Residence Service, o reflexo dos seus ...",Pronto para morar,1,1149572
313,2,"Rua Timbiras, Jurunas",82,3,2,1,None,None,Venha conhecer os diferenciais incríveis desse...,Pronto para morar,1,
362,3,"Avenida Marechal Floriano Peixoto, Centro",80,None,None,None,None,None,Na esquina da Rua Marechal Floriano com Av. Se...,Pronto para morar,1,526316


In [18]:
# Acessar os dados salvos e criar um novo DataFrame sem os duplicados
def get_data_files() -> list:
    data_files = []
    cwd = str(os.getcwd())
    _, _, files = next(os.walk(cwd))
    for file in files:
        if 'data' in file:
            data_files.append(file)
    return data_files

def standard_dict() -> dict:
    return {'Posição': [],
            'Endereço': [],
            'Área': [],
            'Quartos': [],
            'Banheiros': [],
            'Vagas': [],
            'Condomínio': [],
            'IPTU': [],
            'Descrição': [],
            'Status': [],
            'Página': [],
            'Preço': []}

def generate_data() -> pd.DataFrame:
    dicio_vazio = standard_dict()
    memo_df = pd.DataFrame(dicio_vazio)
    data_files = get_data_files()
    
    for data in data_files:
        d = pd.read_csv(data)
        memo_df = pd.concat([memo_df, d])
        
    memo_df = memo_df.iloc[:, :-1]
    memo_df = memo_df.reset_index()
    memo_df = memo_df.drop('index', axis=1)
    memo_df = memo_df.drop_duplicates()
    return memo_df

In [19]:
memo_df = generate_data()
memo_df

,Posição,Endereço,Área,Quartos,Banheiros,Vagas,Condomínio,IPTU,Descrição,Status,Página,Preço
0,1.0,"Rodovia BR-364, Portal da Amazonia",420,1,1,NaN,NaN,NaN,"ECOVILLE- Lindo lote a venda, plano, sol da ma...",Super destaque,1.0,389000.0
1,2.0,"Rodovia BR-364, Portal da Amazonia",429,NaN,NaN,NaN,NaN,NaN,O lote/terreno está localizado no bairro Porta...,Super destaque,1.0,360000.0
2,3.0,"Rua Severina Maria de Sousa e Silva, Jardim Ma...",99,3,2,2,690.0,NaN,O apartamento está localizado no bairro Jardim...,Super destaque,1.0,570000.0
3,4.0,"Estrada do Calafate, Calafate",50,2,1,1,NaN,NaN,Vendo esse imóvel em condomínio fechado. Apart...,Comum,1.0,89000.0
4,5.0,"Rua C, LBA Vila Betel ll",336,2,2,1,NaN,NaN,?Casa na Vila Betel ! Vende-se uma casa no co...,Comum,1.0,185000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5707,20.0,"Rua Chocalho, Castanheira",250,4,3,2,NaN,NaN,VENDE-SE CASA NO BAIRRO CASTANHEIRAS Casa pos...,Destaque,1.0,160000.0
5708,21.0,"Rua Soledade, Marcos Freire",90,2,1,NaN,NaN,NaN,Grande oportunidade para investidor Vila de c...,Comum,1.0,135000.0
5709,22.0,"Rua Getúlio Vargas, São João Bosco",250,4,4,4,NaN,NaN,Vendo esta excelente residencia na cidade de P...,Destaque,1.0,450000.0
5710,23.0,"Rua Jamary, Pedrinhas",120,3,4,2,NaN,NaN,A casa está localizado no bairro Pedrinhas tem...,Comum,1.0,550000.0


In [20]:
# Teste
for i in memo_df.duplicated():
    if i:
        print("Achou duplicado")